## Exercise 1: Implementation of multi-layer NN

#### 1. Initialization

In [4]:
import numpy as np

In [3]:
np.random.seed(666)

In [ ]:
def initialize(K, D, D_i, D_o):
    W = [None] * (K+1)
    B = [None] * (K+1)

    #initialize first layer
    W[0] = np.random.normal(size=(D, D_i))
    B[0] = np.random.normal(size =(D,1))

    #hidden layers
    for layer in range(1,K):
        W[layer] = np.random.normal(0, np.sqrt(2 / D), size=(D, D)) #He initialization
        B[layer] = np.zeros(size=(D,1))

    #last layer
    W[-1] = np.random.normal(size=(D_o, D))
    B[-1]= np.random.normal(size =(D_o,1))


    return W,B

# Number of hidden layers
K = 5
# Number of neurons per layer
D = 6
# Input layer dimension
D_i = 1
# Output layer dimension
D_o = 1

W, B = initialize(K,D, D_i, D_o)


#### 2. Activation functions

In [ ]:
def relu(x):
    activation = x.clip(0.0)
    return activation

def relu_der(x):
    dx = np.array(x)
    dx[dx>0] = 1
    dx[dx<=0] = 0
    return dx

def sigmoid(x):
    activation = 1.0 / (1.0 + np.exp(-x))
    return activation

def sigmoid_der(x):
    sigma = 1.0 / (1.0 + np.exp(-x))
    dx = sigma * (1.0 - sigma)
    return dx